In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations

# Load data
df = pd.read_csv('cities.csv', header=None, names=['city', 'x', 'y'])

# Prompt user for current city
current_city = input("Please enter your current city (leave empty and press enter to use the first city in the list): ")

# If no city is entered, use the first city in the list
if current_city == '':
    current_city = df.iloc[0]['city']
# If a city is entered, validate input
elif current_city not in df['city'].values:
    raise ValueError("Invalid city. Make sure it's one of the cities in the dataset.")

# Calculate distance matrix
distances = distance_matrix(df[['x', 'y']].values, df[['x', 'y']].values)
dist_matrix = pd.DataFrame(distances, index=df.city, columns=df.city)

# Create graph
G = nx.from_numpy_array(dist_matrix.values.astype(float))

# Get index of current city
source_index = df[df['city'] == current_city].index[0]

# Find initial path
tsp_route = nx.approximation.greedy_tsp(G, source=source_index)

# 2-opt algorithm
for _ in range(50):  # Run for a fixed number of iterations
    for i, j in combinations(range(1, len(tsp_route) - 1), 2):  # For each pair of edges in the path
        if i != j:
            new_route = tsp_route[:i] + tsp_route[i:j][::-1] + tsp_route[j:]  # Try reversing the path between i and j
            if sum(dist_matrix.values[new_route[i], new_route[i - 1]] for i in range(1, len(new_route))) < sum(
                    dist_matrix.values[tsp_route[i], tsp_route[i - 1]] for i in range(1, len(tsp_route))):  # If the new path is shorter
                tsp_route = new_route  # Update the path

# Get city names
route = [(df.iloc[i]['city'], df.iloc[i]['x'], df.iloc[i]['y']) for i in tsp_route]

# Remove last city to make it one way
route_one_way = route[:-1]

# Save result
with open('route.txt', 'w') as f:
    for city, x, y in route_one_way:
        f.write(f'{city}, {x}, {y}\n')

# Create plot
plt.figure(figsize=(10, 10))
plt.scatter(df['x'], df['y'])

# Add labels
for _, row in df.iterrows():
    plt.text(row['x'], row['y'], row['city'])

prev_node = tsp_route[0]
for node in tsp_route[1:-1]:  # stop one city before the end
    plt.plot([df.iloc[prev_node]['x'], df.iloc[node]['x']], [df.iloc[prev_node]['y'], df.iloc[node]['y']], 'k-')
    prev_node = node
plt.show()
